In [ ]:
%%capture --no-display

# Librairies utilisées dans compilation_donnees.py
# ! pip install pandas geopandas scipy matplotlib

# Librairies pour l'importation des données
# ! pip install pandas py7xr xlrd openpyxl

# Librairies et setup pour effectuer le scrapping
# ! pip install pytest-playwright playwright_stealth asyncio
# ! sudo playwright install-deps
# ! playwright install

# 1 - Traitement et importation des données

## Données tabulaires

Toutes les fonctions d'extraction de données brutes se trouvent dans le script `scripts/compilation_donnees.py`. Les fonctions `importer_tout()` et `importer_locale(date)` permettent d'importer toutes les sources.

In [1]:
## Importer les données depuis les APIs (plus lent)
# from scripts.importation_donnees_tabuaires import *
# df_SAE, df_SAE_2011, df_dep, df_finess, df_drees, df_pauv = importer_tout() 

## Importer une sauvegarde locale récente des données
from scripts.compilation_donnees import *
df_SAE, df_SAE_2011, df_dep, df_finess, df_drees, df_pauv, df_doctolib = importer_locale("2025-12-16-17-35", "2025-11-20-14-56")

Travail sur les données DREES pour réaliser des évolutions dans le temps au niveau national : juste présenter les données comme y'a déjà publier sur la DREES 

In [ ]:
df_drees_filtre = df_drees[df_drees["zone_geo"] == "France entière"]
df_drees_filtre.plot(x="annee", y=["TOT_IVG", "IVG_HOSP_INS", "IVG_HOSP_MED"], kind="line", marker='o', figsize=(10,6), title="Évolution nombre total d'IVG en France de 2016 à 2024")

La hausse du nombre d’IVG depuis 2022 se poursuit, après la nette baisse en 2020 et 2021 en lien avec la pandémie de Covid-19.
Le nombre d’IVG a tendance a augmenté depuis 2017. Cette hausse est interrompu en 2020 et 2021, où le nombre d’IVG connait une chute en lien avec la pandémie de Covid-19. Depuis 2022, ce chiffre augmente de nouveau, et plus rapidement que précédemment. 
Le nombre d’IVG tardive suit une évolution similaire. Néanmoins, la hausse est plus faible que celle du nombre d’IVG totale entre 2023 et 2024, ce qui peut indiquer un changement des pratiques, qui suit le changement législatif de 2022. En effet, depuis 2022, il est légal de recourir à une IVG après 14 semaines de grossesse, deux semaines de plus qu’auparavant. 

In [2]:
df = jointure(df_SAE, df_SAE_2011, df_dep, df_finess, df_drees, df_pauv, df_doctolib)
df.head(5)

,code_dep,département,femmes,pris,conv,centre_de_sante_sexuelle,cegidd,centres_total,hopitaux_sans_ivg_tard,ivg_hosp_ins,...,age_35_39,age_40&plus,ivg_go,ivg_mg,ivg_sf,ivg_aut,taux_pauvrete,doctolib_trimestre,doctolib_mois,doctolib_2sem
0,01,Ain,289335,5.0,1.0,1.0,2.0,3.0,2.0,375.0,...,332.0,127.0,0,88,288,NC,10.8,88.0,78.0,70.0
1,02,Aisne,207602,5.0,4.0,5.0,3.0,8.0,NaN,313.0,...,274.0,111.0,NC,NC,82,0,18.8,6.0,5.0,5.0
2,03,Allier,117763,4.0,5.0,3.0,3.0,6.0,1.0,149.0,...,161.0,64.0,NC,NC,409,0,16.2,9.0,9.0,7.0
3,04,Alpes-de-Haute-Provence,59730,2.0,0.0,0.0,0.0,0.0,NaN,110.0,...,112.0,36.0,0,NC,NC,0,17.1,10.0,9.0,6.0
4,05,Hautes-Alpes,51076,2.0,22.0,2.0,0.0,2.0,NaN,87.0,...,104.0,54.0,NC,270,NC,0,14.7,11.0,11.0,9.0


In [3]:
df_norm = normalisation(df)
df_norm.head(5)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
valeur_min = df["femmes"].min()

print(valeur_min) # 27982 < 100 000 : donc on conserve pr avoir un chiffre interprétable mais pas réaliste pr certains départements

## Web Scrapping - Doctolib

Pour avoir un **indice de la disponibilité des IVGs en France**, nous allons utiliser des données issues du site de prise de rendez-vous médicales le plus utilisé en France, **Doctolib**. Par département et spécialisation IVG, les données extraites sont le nombre de localisations différentes proposant un RDV IVG dans un délai de 14, 30 et 90 jours. Ces données sont accessibles à tous gratuitement en utilisant leur moteur de recherche et des filtres (par [ex](https://www.doctolib.fr/ivg-medicamenteuse/paris?availabilitiesBefore=90&regulationSector%5B%5D=CONTRACTED_1&regulationSector%5B%5D=CONTRACTED_1_WITH_EXTRA&regulationSector%5B%5D=CONTRACTED_1_WITH_OPTAM&regulationSector%5B%5D=CONTRACTED_1_WITH_OPTAM_CO&regulationSector%5B%5D=CONTRACTED_2&regulationSector%5B%5D=CONTRACTED_2_WITH_OPTAM&regulationSector%5B%5D=CONTRACTED_2_WITH_OPTAM_CO&regulationSector%5B%5D=CONTRACTED&regulationSector%5B%5D=CONTRACTED_WITH_EXTRA&regulationSector%5B%5D=ORGANIZATION_CONTRACTED)).

Pour la mise en oeuvre, le script est `scrapper_doctolib.py` qui crée un .csv à l'aboutissement. Le script s'appuie sur la librairie `Playwright` permet de faire du web scrapping sur des sites dynamiques (comme Doctolib qui utilise react et javascript) en utilisant des navigateurs "headless" dans l'arrière plan. Avant de finir sur cette librairie, d'autres options ont été tentées : `BeautifulSoup` (pour adapté au parsing des sites statiques) et `Selenium` (une autre librairie de "headless browsers"). Cependant, `Playwright` a était le plus agile pour la gestion du chargement de la page. De plus, `asyncio` est utilisé pour faire du parallélisme et réduire le temps de scrapping.

> Point législatif
> 
> [Source (CNIL) : ](https://www.cnil.fr/fr/focus-interet-legitime-collecte-par-moissonnage)

Exemple des données obtenues (format dataframe pandas) suite au scraping (environ une demi-heure) :

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
url = "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb" 
geo = gpd.read_file(url)
geo['code_dep'] = geo['code'].astype(str).str.zfill(2)
gdf = pd.merge(df_doctolib, geo, on='code_dep', how='left')

In [ ]:
# import geopandas as gpd
# import folium
# import mapclassify
# import matplotlib

# gdf_aggregated = gpd.GeoDataFrame(aggregated)

# gdf_aggregated.explore(
#     column="trimestre",
#     scheme="quantiles", 
#     legend=True,
#     tooltip=["nom", "trimestre", "mois", "deuxSemaines"],
#     cmap="Reds",
#     tiles="CartoDB positron",
#     k=10,                 
#     legend_kwds=dict(colorbar=True)
# )


Cette carte présente les taux de disponibilité d’IVG dans un délai de moins de 3 mois via doctolib dans un département. Les disparités territoriales sont importantes. Certains départements comme l’Ile-de-France et …. ont une plus grande offre de rendez-vous médicaux pour réaliser une IVG, quelque soit le professionnel de santé. D’autres, comme …, ont seulement … offre pour 1000 femmes. Ces constats révèlent des offres différenciées selon les départements, et donc des inégalités territoriales pour recourir à une IVG. 

In [ ]:
# gdf_aggregated = gpd.GeoDataFrame(gdf_aggregated)

# gdf_aggregated.explore(
#     column="trimestre_norm",
#     scheme="quantiles", 
#     legend=True,
#     tooltip=["nom", "trimestre_norm", "mois_norm", "deuxSemaines_norm"],
#     cmap="Reds",
#     tiles="CartoDB positron",
#     k=10,                 
#     legend_kwds=dict(colorbar=True)
# )


In [ ]:
main = pd.merge(gdf_aggregated, df_dep_ivg, on='code_dep', how='outer')

In [ ]:
main.head()